# Limpieza informacion usuario

Este código limpia los datos que tiene Giganav de sus usuarios y los distribuye para ser utilizado para llamadas.

Importo repositorios

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import matplotlib.ticker as mtick
import os
import platform

Establezco directorios

In [2]:
if platform.system()=='Windows': 
    #Directorios Windows
    os.chdir(r"G:\.shortcut-targets-by-id\120bUa35y3FJzvumWahpOx1qS9I6TIrhb\GIGANAV")
    folder=os.getcwd()
    data=folder+'\\'+'Datos'
    facturas=data+'\\'+'Facturas'
    tickets=data+'\\'+'Tickets'
    usuarios=data+'\\'+'Usuarios'
    egresos=data+'\\'+'Egresos'
    transacciones=data+'\\'+'Transacciones'
    analisis=folder+'\\'+'Operacion\\Analisis'
    bases=analisis+'\\'+'Bases_analisis'
    outputs=analisis+'\\'+'Output'
    plots=outputs+'\\'+'Gráficos'
    usuarios_clean=bases+'\\'+'Usuarios'
    transac_clean=bases+'\\'+'Transacciones'
    tickets_clean=bases+'\\'+'Tickets'
    facturas_clean=bases+'\\'+'Facturas'
    egresos_clean=bases+'\\'+'Egresos'
    
if platform.system()=='Darwin':
    #Directorios Mac
    folder='/Users/juliandiazparra/Library/CloudStorage/GoogleDrive-julian.adiaz@hotmail.com/Mi unidad/Negocios/Familiar/GIGANAV'
    data=folder+'/Datos'
    facturas=data+'/Facturas'
    tickets=data+'/Tickets'
    usuarios=data+'/Usuarios'
    egresos=data+'/Egresos'
    transacciones=data+'/Transacciones'
    analisis=folder+'/Operacion/Analisis'
    bases=analisis+'/Bases_analisis'
    outputs=analisis+'/Outputs'
    plots=outputs+'/Gráficos'
    usuarios_clean=bases+'/Usuarios'
    transac_clean=bases+'/Transacciones'
    tickets_clean=bases+'/Tickets'
    facturas_clean=bases+'/Facturas'
    egresos_clean=bases+'/Egresos'

## Importo planes y velocidades

In [3]:
#Importo planes
import glob
planes_cd=os.path.join(data,'Planes')
file_list=glob.glob(os.path.join(planes_cd,'*.csv'))
last_file=max(file_list,key=os.path.getctime)
base_planes=pd.read_csv(last_file)
base_planes.head()

,ID,Nombre,Descarga Kbps,Subida Kbps,Profile,Precio,Activos,Suspendidos,Unnamed: 8
0,1,GIGABASIC,6144,6144,NaN,"COL$ 50.000,00",4,1,NaN
1,2,GIGAMAX,8192,8192,NaN,"COL$ 70.000,00",1,0,NaN
2,3,GIGAXTREM,10240,10240,NaN,"COL$ 90.000,00",0,1,NaN
3,7,GIGAPONAL,10240,4096,NaN,"COL$ 100.000,00",3,0,NaN
4,16,GIGAMAX+,8192,8192,NaN,"COL$ 70.000,00",1,0,NaN


Creo variable de megas con base en datos de base de planes

In [4]:
#Clasifico numero de megas por plan
def regla_personalizada(valor):
    if valor == 105:
        return 100
    elif valor == 11:
        return 10
    elif valor == 16:
        return 15
    elif valor == 26:
        return 25
    elif valor == 31:
        return 30
    elif valor == 41:
        return 40
    elif valor == 51:
        return 50
    elif valor == 61:
        return 60
    elif valor == 101:
        return 100
    elif valor == 151:
        return 150
    else:
        return valor

base_planes.loc[:,
    'megas']=np.floor(base_planes['Descarga Kbps'].astype(int)/1000).apply(regla_personalizada)

base_planes['megas'].value_counts()

megas
100.0    7
10.0     6
8.0      3
15.0     3
25.0     3
30.0     3
40.0     3
50.0     3
60.0     3
150.0    3
6.0      2
200.0    2
300.0    2
500.0    2
Name: count, dtype: int64

In [5]:
df_planes_clean=base_planes.loc[:,['ID','Nombre','megas']]
df_planes_clean.columns=['id_plan','Plan','megas']

## Importo datos de usuarios

In [6]:
import glob
list_of_files = glob.glob(os.path.join(usuarios,'Lista de Usuarios*.csv'))
latest_file = max(list_of_files, key=os.path.getctime)
base_usuarios=pd.read_csv(latest_file, low_memory=False)

import_timestamp_users=os.path.getctime(latest_file)
import_date_users=datetime.fromtimestamp(import_timestamp_users).strftime('%y_%m_%d')
base_usuarios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3893 entries, 0 to 3892
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           0 non-null      float64
 1   Cedula               3891 non-null   object 
 2   Id                   3893 non-null   int64  
 3   Mac                  113 non-null    object 
 4   Nombre               3893 non-null   object 
 5   Instalado            2281 non-null   object 
 6   Dirección Principal  3866 non-null   object 
 7   Deuda actual         1361 non-null   object 
 8   Plan voip            1327 non-null   object 
 9   Plan                 2281 non-null   object 
 10  Ip                   2281 non-null   object 
 11  Tipo estrato         3893 non-null   int64  
 12  Fecha suspendido     3893 non-null   object 
 13  Ultimo pago          3816 non-null   object 
 14  Correo               2958 non-null   object 
 15  Telefono             2919 non-null   o

Me quedo con información relevante

In [7]:
depurada_base_usuarios=base_usuarios.loc[:,[
    'Id', 'Nombre', 'Plan voip', 
    'Plan', 'Deuda actual', 'Tipo estrato', 
    'Correo', 'Telefono', 'Movil', 
    'Zona', 'Total cobrar']]
depurada_base_usuarios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3893 entries, 0 to 3892
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Id            3893 non-null   int64 
 1   Nombre        3893 non-null   object
 2   Plan voip     1327 non-null   object
 3   Plan          2281 non-null   object
 4   Deuda actual  1361 non-null   object
 5   Tipo estrato  3893 non-null   int64 
 6   Correo        2958 non-null   object
 7   Telefono      2919 non-null   object
 8   Movil         3824 non-null   object
 9   Zona          3866 non-null   object
 10  Total cobrar  3893 non-null   object
dtypes: int64(2), object(9)
memory usage: 334.7+ KB


In [8]:
depurada_base_usuarios.head()

,Id,Nombre,Plan voip,Plan,Deuda actual,Tipo estrato,Correo,Telefono,Movil,Zona,Total cobrar
0,3376,Prueba Prueba RETIRADO,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,"COL$ 0,00"
1,3378,PRUEBA GIGANAV RETIRADO,NaN,NaN,NaN,1,NaN,NaN,NaN,VILLA FANNY,"COL$ 0,00"
2,1374,RODRIGO SILVA SEPULVEDA ACTIVO,NaN,GIGABASIC RURAL,NaN,1,silva_rodrigo_25@hotmail.com,3206671131,3153598524,CGTO MINAS,"COL$ 120.000,00"
3,2450,PUNTO GRATIS ACTIVO,NaN,30X30X70 GPON3,NaN,1,NaN,NaN,NaN,NaN,"COL$ 0,00"
4,788,JHON FREDY FONSECA CARRANZA RETIRADO,NaN,NaN,"3 COL$ 182.000,00",1,jhonfredyfonsecacarranza@gmail.com,3162662078,3187613025,CGTO LA LLANA,"COL$ 0,00"


Creo categorias de usuarios y limpio nombre

In [9]:
# Lista de categorías de usuarios
categorias_usuarios = ['activo', 'retirado', 'suspendido']

# Crear estado del usuario en una sola operación
for estado in categorias_usuarios:
    depurada_base_usuarios.loc[depurada_base_usuarios[
    'Nombre'].str.contains(estado, case=False), 'estado'] = estado

# Crear nombre corregido eliminando las palabras 'activo', 'retirado' y 'suspendido'
depurada_base_usuarios['nombre'] = depurada_base_usuarios['Nombre'].str.replace(r'(?i)(activo|retirado|suspendido)', '', regex=True).str.strip()

# Opcional: Convertir nombres a título (capitalizar correctamente)
depurada_base_usuarios['nombre'] = depurada_base_usuarios['nombre'].str.title()

# Verificar resultado
print(depurada_base_usuarios[['Nombre', 'estado', 'nombre']].head())

                                  Nombre    estado  \
0                Prueba Prueba  RETIRADO  retirado   
1              PRUEBA  GIGANAV  RETIRADO  retirado   
2        RODRIGO SILVA SEPULVEDA  ACTIVO    activo   
3                  PUNTO GRATIS   ACTIVO    activo   
4  JHON FREDY FONSECA CARRANZA  RETIRADO  retirado   

                        nombre  
0                Prueba Prueba  
1              Prueba  Giganav  
2      Rodrigo Silva Sepulveda  
3                 Punto Gratis  
4  Jhon Fredy Fonseca Carranza  


In [10]:
categorias_usuarios=['activo', 'retirado', 'suspendido']

for j, i in enumerate(categorias_usuarios):
    #Creo estado del usuario
    depurada_base_usuarios.loc[
        depurada_base_usuarios['Nombre'].str.contains(i, case=False), 
        'estado']=categorias_usuarios[j]
    
#Creo nombre corregido
depurada_base_usuarios.loc[:, 'nombre']=depurada_base_usuarios[
    'Nombre'].str.replace('activo', '', case=False)
depurada_base_usuarios.loc[:, 'nombre']=depurada_base_usuarios[
    'nombre'].str.replace('retirado', '', case=False)
depurada_base_usuarios.loc[:, 'nombre']=depurada_base_usuarios[
    'nombre'].str.replace('suspendido', '', case=False)

Agrego planes, megas y variable TV

In [11]:
#Pego base planes y megas
depurada_base_usuarios=depurada_base_usuarios.merge(df_planes_clean,on='Plan',how='left')

# Llenar valores NaN de 'Plan voip'
depurada_base_usuarios.loc[:,'Plan voip'] = depurada_base_usuarios['Plan voip'].fillna('')

# Crear la variable 'TV'
depurada_base_usuarios.loc[:,'TV'] = ((depurada_base_usuarios['Plan'].str.contains('plus',case=False))|
                                (depurada_base_usuarios['Plan voip'].str.contains('recaudo a terceros', case=False))).astype(int)

In [12]:
#Verifico segmentos de la empresa por megas y TV
pd.pivot_table(
    depurada_base_usuarios, index='megas', columns='TV', values='Id', aggfunc='nunique')

TV,0,1
megas,,
6.0,97.0,NaN
8.0,9.0,NaN
10.0,60.0,100.0
15.0,20.0,78.0
25.0,14.0,28.0
30.0,19.0,28.0
40.0,4.0,18.0
50.0,209.0,389.0
60.0,22.0,50.0


Creo variable radio

In [13]:
#Lleno vacios en plan
depurada_base_usuarios['Plan']=depurada_base_usuarios['Plan'].fillna('')

# Inicializar la columna 'radio' con 0 (suponiendo que es de fibra óptica por defecto)
depurada_base_usuarios['radio'] = 0

# Actualizar la columna 'radio' a 1 para los usuarios con planes de radioenlace (rural)
depurada_base_usuarios.loc[depurada_base_usuarios['Plan'].str.contains('GIGA', case=False) & 
                           ~depurada_base_usuarios['Plan'].str.contains(' VIP | ORO | PLATA | BRONCE', case=False), 'radio'] = 1

# Verificar los resultados
print(depurada_base_usuarios[['Plan', 'radio']].value_counts())

Plan                              radio
                                  0        1612
GIGA VIP PLUS 100M SIMETRICAS     0         626
GIGA VIP 100M SIMETRICAS          0         325
50X50X150 GPON3                   0         313
50X50X150 GPON2                   0         199
GIGABASIC RURAL                   1          92
50X50X150 GPON1                   0          86
GIGA BRONCE PLUS 200M SIMETRICAS  0          72
10X10X50 GPON3                    0          55
10X10X50 GPON1                    0          54
15X15X55 GPON3                    0          47
10X10X50 GPON2                    0          43
100X100X75 GPON 3                 0          34
15X15X55 GPON2                    0          34
GIGA BRONCE 200M SIMETRICAS       0          27
60X60X60 GPON 3                   0          27
60X60X60 GPON 2                   0          25
30X30X70 GPON3                    0          21
60X60X60 GPON 1                   0          20
15X15X55 GPON1                    0          17


In [14]:
# Crear la columna 'tarifa' limpiando y convirtiendo 'Total cobrar'
depurada_base_usuarios['tarifa'] = (
    depurada_base_usuarios['Total cobrar']
    .str.strip('COL$ ')
    .str.replace('.', '', regex=False)  # Eliminar puntos
    .str.replace(',', '.', regex=False)  # Reemplazar coma por punto
    .astype(float)  # Convertir a tipo float
)

# Ajustar la tarifa si el usuario tiene TV
depurada_base_usuarios.loc[
    depurada_base_usuarios['Plan voip'].str.contains('recaudo',case=False), 'tarifa'] += 25000

In [15]:
# Limpiar y crear la columna 'meses_suspendido' para usuarios suspendidos
depurada_base_usuarios.loc[:, 'meses_suspendido'] = depurada_base_usuarios.loc[
    depurada_base_usuarios['estado'] == 'suspendido',
    'Deuda actual'].str.split(' ', expand=True, n=1).iloc[:, 0].fillna(0).astype('int')

# Limpiar y crear la columna 'deuda' para usuarios suspendidos
depurada_base_usuarios.loc[:, 'deuda'] = depurada_base_usuarios.loc[
    depurada_base_usuarios['estado'] == 'suspendido',
    'Deuda actual'].str.split(' ', expand=True, n=1).iloc[:, 1].str.strip(
    'COL$ ').str.replace('.', '', regex=False).str.replace(
    ',00', '', regex=False).str.lstrip().str.replace(',', '.').fillna(0).astype('float')

In [16]:
depurada_base_usuarios.loc[depurada_base_usuarios['estado']=='activo']

,Id,Nombre,Plan voip,Plan,Deuda actual,Tipo estrato,Correo,Telefono,Movil,Zona,Total cobrar,estado,nombre,id_plan,megas,TV,radio,tarifa,meses_suspendido,deuda
2,1374,RODRIGO SILVA SEPULVEDA ACTIVO,,GIGABASIC RURAL,NaN,1,silva_rodrigo_25@hotmail.com,3206671131,3153598524,CGTO MINAS,"COL$ 120.000,00",activo,RODRIGO SILVA SEPULVEDA,18.0,6.0,0,1,120000.0,NaN,NaN
3,2450,PUNTO GRATIS ACTIVO,,30X30X70 GPON3,NaN,1,NaN,NaN,NaN,NaN,"COL$ 0,00",activo,PUNTO GRATIS,41.0,30.0,0,0,0.0,NaN,NaN
6,1722,MARIA ALEJANDRA RESTREPO MORALES ACTIVO,,GIGA VIP PLUS 100M SIMETRICAS,NaN,1,Marymkery124@gmail.com,3128227880,3105378643,LOS PINOS,"COL$ 80.000,00",activo,MARIA ALEJANDRA RESTREPO MORALES,63.0,100.0,1,0,80000.0,NaN,NaN
7,3806,2-MARIA ALEJANDRA RESTREPO MORALES ACTIVO,,GIGA VIP PLUS 100M SIMETRICAS,NaN,1,marymkery124@gmail.com,3123140321,3105378643,1º DE MAYO,"COL$ 70.000,00",activo,2-MARIA ALEJANDRA RESTREPO MORALES,63.0,100.0,1,0,70000.0,NaN,NaN
10,4176,LUIS ALFREDO MEZA MONTIEL ACTIVO,,50X50X150 GPON3,NaN,1,luisalfredomezamontiel1987@gmail.com,3166472810,3185066065,CGTO LA LLANA,"COL$ 50.000,00",activo,LUIS ALFREDO MEZA MONTIEL,43.0,50.0,0,0,50000.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3882,2380,2-FREDY PEREZ MORA ACTIVO,,50X50X150 GPON2,NaN,1,fredyperezmora1982@gmail.com,3188868005,3172972735,CGTO LA PEDREGOSA,"COL$ 50.000,00",activo,2-FREDY PEREZ MORA,34.0,50.0,0,0,50000.0,NaN,NaN
3886,3879,JOSE LUIS PEREZ MONTERO ACTIVO,,60X60X60 GPON 3,NaN,1,jperez.jlpm24@gmail.com,3122140668,3107291306,SAN RAFAEL,"COL$ 60.000,00",activo,JOSE LUIS PEREZ MONTERO,49.0,60.0,0,0,60000.0,NaN,NaN
3888,21,ODAIR MORA VALBUENA ACTIVO,,GIGA VIP 100M SIMETRICAS,NaN,2,odairmoravalbuena@gmail.com,NaN,3152180728,NUEVO HORIZONTE,"COL$ 52.000,00",activo,ODAIR MORA VALBUENA,62.0,100.0,0,0,52000.0,NaN,NaN
3890,1038,JORGE ALBERTO MORENO ACTIVO,,50X50X150 GPON2,NaN,1,NaN,3142082576,3188925978,CGTO LA PEDREGOSA,"COL$ 52.000,00",activo,JORGE ALBERTO MORENO,34.0,50.0,0,0,52000.0,NaN,NaN


In [17]:
variables=['Id', 'nombre', 'estado', 
           'radio', 'Tipo estrato', 'megas', 
           'TV', 'Telefono', 'Movil', 
           'Correo', 'Zona', 'tarifa', 
           'meses_suspendido', 'deuda']
base_final_contactos=depurada_base_usuarios.loc[:,variables]
base_final_contactos[['meses_suspendido','deuda']]=base_final_contactos[['meses_suspendido','deuda']].fillna(0)
base_final_contactos

,Id,nombre,estado,radio,Tipo estrato,megas,TV,Telefono,Movil,Correo,Zona,tarifa,meses_suspendido,deuda
0,3376,Prueba Prueba,retirado,0,1,NaN,0,NaN,NaN,NaN,NaN,0.0,0.0,0.0
1,3378,PRUEBA GIGANAV,retirado,0,1,NaN,0,NaN,NaN,NaN,VILLA FANNY,0.0,0.0,0.0
2,1374,RODRIGO SILVA SEPULVEDA,activo,1,1,6.0,0,3206671131,3153598524,silva_rodrigo_25@hotmail.com,CGTO MINAS,120000.0,0.0,0.0
3,2450,PUNTO GRATIS,activo,0,1,30.0,0,NaN,NaN,NaN,NaN,0.0,0.0,0.0
4,788,JHON FREDY FONSECA CARRANZA,retirado,0,1,NaN,0,3162662078,3187613025,jhonfredyfonsecacarranza@gmail.com,CGTO LA LLANA,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3888,21,ODAIR MORA VALBUENA,activo,0,2,100.0,0,NaN,3152180728,odairmoravalbuena@gmail.com,NUEVO HORIZONTE,52000.0,0.0,0.0
3889,2875,ALIRIO DE JESUS MONSALVE OSPINA,retirado,0,1,NaN,0,3126429826,3213262547,aliriooodejesusmonsalve1805@gmail.com,CGTO LA PEDREGOSA,0.0,0.0,0.0
3890,1038,JORGE ALBERTO MORENO,activo,0,1,50.0,0,3142082576,3188925978,NaN,CGTO LA PEDREGOSA,52000.0,0.0,0.0
3891,3265,JESUS MARIA HERNANDEZ HENAO,retirado,0,1,NaN,0,3145291879,3208370958,jesusmariahernan@gmail.com,CGTO LA PEDREGOSA,25000.0,0.0,0.0


## Base contactos Hubspot

In [18]:
list_of_files = glob.glob(os.path.join(usuarios,'Usuarios_export_Hubspot*.csv'))
latest_file = max(list_of_files, key=os.path.getctime)
base_hubspot=pd.read_csv(latest_file, low_memory=False)
base_hubspot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3956 entries, 0 to 3955
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID de registro           3956 non-null   int64  
 1   Nombre                   3950 non-null   object 
 2   Apellidos                17 non-null     object 
 3   estado suscripción       3924 non-null   object 
 4   Tarifa                   3924 non-null   float64
 5   Megas                    3924 non-null   float64
 6   TV                       3924 non-null   float64
 7   Telefono                 2973 non-null   object 
 8   Movil                    3858 non-null   object 
 9   Correo de miembro        2951 non-null   object 
 10  Última actividad         502 non-null    object 
 11  ID_mikrowisp             3924 non-null   float64
 12  Zona                     3897 non-null   object 
 13  Etapa del ciclo de vida  3956 non-null   object 
 14  meses_suspendido        

In [19]:
#Cambio nombre y hago merge para pegar solo el Record ID de Hubspot
#Elimino duplicados y me quedo con el record de hubspot mas reciente
base_hubspot.rename(columns={
    'ID_mikrowisp': 'Id'}, inplace=True)

base_hubspot=base_hubspot.groupby(
        'Id').agg(
        id_hubspot=(
        'ID de registro', 'min')).reset_index()

base_hubspot.rename(columns={
    'id_hubspot': 'ID de registro'}, inplace=True)

#Hago merge
base_records_hubspot=base_final_contactos.merge(base_hubspot.loc[
    :,['Id', 'ID de registro']], on='Id', how='left')

base_records_hubspot.loc[
    :, 'ID de registro']=base_records_hubspot[
    'ID de registro'].astype('float')

base_records_hubspot.head()

,Id,nombre,estado,radio,Tipo estrato,megas,TV,Telefono,Movil,Correo,Zona,tarifa,meses_suspendido,deuda,ID de registro
0,3376,Prueba Prueba,retirado,0,1,NaN,0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,21911.0
1,3378,PRUEBA GIGANAV,retirado,0,1,NaN,0,NaN,NaN,NaN,VILLA FANNY,0.0,0.0,0.0,20907.0
2,1374,RODRIGO SILVA SEPULVEDA,activo,1,1,6.0,0,3206671131,3153598524,silva_rodrigo_25@hotmail.com,CGTO MINAS,120000.0,0.0,0.0,25003.0
3,2450,PUNTO GRATIS,activo,0,1,30.0,0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,35701.0
4,788,JHON FREDY FONSECA CARRANZA,retirado,0,1,NaN,0,3162662078,3187613025,jhonfredyfonsecacarranza@gmail.com,CGTO LA LLANA,0.0,0.0,0.0,25405.0


In [20]:
print('El número de usuarios suspendidos al '+import_date_users+' es '+str(base_records_hubspot.loc[
    base_records_hubspot['estado']=='suspendido', 'Id'].groupby(level=0).nunique().sum()))

El número de usuarios suspendidos al 24_10_08 es 320


Exporto base para hubspot

In [21]:
filepath=os.path.join(usuarios_clean,'Base_usuarios_hub_'+import_date_users)
base_records_hubspot.to_csv(filepath+'.csv', index=False)

##### 